In [6]:
import pandas as pd

course_descriptions_df = pd.read_json("llm_course_descriptions.jsonl", lines=True)
course_descriptions_infer_df = pd.read_json(
    "llm_course_descriptions-inference.jsonl", lines=True
)


def get_course_id_from_name(course_name):
    return courses[courses["name"] == course_name]["course_id"].values[0]


import sqlite3

conn = sqlite3.connect("roster_reviews.sqlite.db")

descriptions = pd.read_sql_query("SELECT * FROM course_descriptions", conn)
courses = pd.read_sql_query("SELECT * FROM courses", conn)

conn.close()

In [8]:
test_id = get_course_id_from_name("AAP 1100")
course_id = test_id

len(
    course_descriptions_infer_df[
        course_descriptions_infer_df["course_id"] == course_id
    ]["response"].values
)


def get_pre_reqs_from_id(id):
    for c in course_descriptions_df[course_descriptions_df["course_id"] == id][
        "response"
    ].values[0]["hard_written_prerequisites"]:
        d = c.split(" ")[0]
        code = c.split(" ")[1]
        yield f"{d} {code}"


def get_all_pre_reqs(id):
    frontier = set([id])
    visited = set()
    while frontier:
        current = frontier.pop()
        visited.add(current)
        for pre_req in get_pre_reqs_from_id(current):
            pre_req_id = get_course_id_from_name(pre_req)
            if pre_req_id not in visited:
                frontier.add(pre_req_id)
    return reversed(list(visited))

1